In [1]:
import pandas as pd

# CNN classifier results for CARS

CNN pretrained on ImageNet predicted various classes from that dataset. Now, we manually classify these classes into cars or not cars.

In [2]:
path_to_csv = 'datasets_generated_ready_models/stylegan2/classifier_predictions_top_1_cars.csv'
df_stylegan2_cars = pd.read_csv(path_to_csv)
df_stylegan2_cars['prediction'].unique()

array(['beach wagon, station wagon, wagon, estate car, beach waggon, station waggon, waggon',
       'car wheel', 'racer, race car, racing car',
       'cab, hack, taxi, taxicab', 'grille, radiator grille',
       'pickup, pickup truck', 'convertible', 'sports car, sport car',
       'Model T', 'television, television system', 'minivan',
       'limousine, limo', 'motor scooter, scooter', 'library',
       'jeep, landrover', 'minibus', 'maillot', 'moving van', 'go-kart',
       'trailer truck, tractor trailer, trucking rig, rig, articulated lorry, semi',
       'carousel, carrousel, merry-go-round, roundabout, whirligig',
       'snowmobile', 'bobsled, bobsleigh, bob', 'half track',
       'tow truck, tow car, wrecker', 'ambulance', 'harvester, reaper',
       'golfcart, golf cart', 'web site, website, internet site, site',
       'street sign'], dtype=object)

In [3]:
# hand-picked class names from the predictions that represent cars or are similar

car_names = ['seat belt, seatbelt,jeep, landrover',
 'sports car, sport car',
 'pickup, pickup truck',
 'convertible',
 'school bus',
 'limousine, limo',
 'streetcar, tram, tramcar, trolley, trolley car',
 'minivan',
 'recreational vehicle, RV, R.V.',
 'racer, race car, racing car',
 'Model T',
 'tow truck, tow car, wrecker',
 'recreational vehicle, RV, R.V.,minibus',
 'beach wagon, station wagon, wagon, estate car, beach waggon, station waggon, waggon',
 'school bus,golfcart, golf cart',
 'minibus',
 'fire engine, fire truck',
 'moving van',
 'jeep, landrover',
 'police van, police wagon, paddy wagon, patrol wagon, wagon, black Maria',
 'trailer truck, tractor trailer, trucking rig, rig, articulated lorry, semi',
 'cab, hack, taxi, taxicab',
 'go-kart']

In [4]:
df_stylegan2_cars['classified_as_car'] = df_stylegan2_cars.prediction.apply(lambda x: 'car' if x in car_names else 'other')
cars_in_images_stylegan2 = (df_stylegan2_cars['classified_as_car'] == 'car').sum()

print("stylegan2")
print(f"Total images: {len(df_stylegan2_cars)}")
print(f"Images classified as cars: {cars_in_images_stylegan2}")
print(f"Percentage of images classified as cars: {cars_in_images_stylegan2/len(df_stylegan2_cars)*100:.2f}%")

stylegan2
Total images: 200
Images classified as cars: 149
Percentage of images classified as cars: 74.50%


# Hypothesis testing

In [5]:
Sn_stylegan2_cars = cars_in_images_stylegan2/len(df_stylegan2_cars)

Sn_stylegan2_cars

0.745

In [7]:
import numpy as np

def add_results_row(results, experiment_name, N, Sn, p0_to_test=[0.5, 0.6, 0.7, 0.8, 0.9]):
    results["Method"].append("CNN")
    results["Experiment"].append(experiment_name)
    results["N"].append(N)
    results["Sn"].append(Sn)
    for p0 in p0_to_test:
        if Sn <= p0:
            results[f"p0={p0}"].append(0)
            continue
        delta = np.exp(-2*N*(Sn-p0)**2)
        results[f"p0={p0}"].append(round(1-delta, 3))
    p0_for_CI_095 = Sn - np.sqrt(np.log(0.05)/(-2*N))
    results["p0 for confidence=0.95"].append(round(p0_for_CI_095, 3))

results = {
    "Method": [],
    "Experiment": [],
    "N": [],
    "Sn" : [],
    "p0=0.5": [],
    "p0=0.6": [],
    "p0=0.7": [],
    "p0=0.8": [],
    "p0=0.9": [],
    "p0 for confidence=0.95": [],
}

add_results_row(results, "stylegan2 cars", len(df_stylegan2_cars), Sn_stylegan2_cars)

df_results = pd.DataFrame(results)
df_results.to_csv("confidence_matrix_CNN_stylegan2.csv", index=False)
df_results

,Method,Experiment,N,Sn,p0=0.5,p0=0.6,p0=0.7,p0=0.8,p0=0.9,p0 for confidence=0.95
0,CNN,stylegan2 cars,200,0.745,1.0,1.0,0.555,0,0,0.658
